# Classification with Deep Learning (CNNs) with a custom Data Generator

In this lesson, we learn how to solve a classification problem through a **Deep Learning** approach based on *Convolutional Neural Networks* (CNNs) and a **custom data generator** to load the data. In the final part, we'll see how is possible to load a single image and **obtain the CNN prediction** (useful for real-worl applications).

**It is absolutely recommended to read the documentation relating to the functions and methods used!**
Usually, it is sufficient typing on Google the name of the function (and eventually the name of the library used).

Let's import some **libraries**. We will use *TensorFlow* and *Keras* as Deep Learning framework.

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt

### Data


1.   Upload the `.zip` file containing the *Euclid* dataset (Deep Learning version!) → this is for **training**
2.   Unzip all files using the following commands. Dataset folders will appear in `/content`

In [ ]:
!unzip -q Euclid_dataset_DL.zip -d /content

### Custom Data Loader

To define our **custom data loader**, we use the object `tf.keras.utils.Sequence`([link](https://www.tensorflow.org/api_docs/python/tf/keras/utils/Sequence)).

Every Sequence must implement the `__getitem__` and the `__len__` methods. If you want to modify your dataset between epochs you may implement `on_epoch_end`. The method `__getitem__` should return a complete batch.




In [ ]:
import numpy as np
import tensorflow as tf
import cv2

class CustomDataGen(tf.keras.utils.Sequence):

    def __init__(self, data_list, num_classes, batch_size, input_size=(224, 224, 3), shuffle=True):

        self.data_list = data_list
        self.num_classes = num_classes
        self.batch_size = batch_size
        self.input_size = input_size
        self.shuffle = shuffle
        self.n = len(self.data_list)

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.data_list)

    # here, we get the input (load an image)
    def __get_input(self, path, target_size):
        img = cv2.imread(path, 1)
        img = img[..., ::-1]
        img = cv2.resize(img, (target_size[0], target_size[1]))
        img = np.asarray(img, dtype=np.float32)
        return img / 255.0

    # here, we get the class number
    def __get_class_number(self, name):
        if 'triangle' in name:
            return 0
        elif 'square' in name:
            return 1
        elif 'rectangle' in name:
            return 2
        elif 'rhombus' in name:
            return 3
        else:
            raise NotImplementedError('Not existing class!')

    # here, we convert the class number in a one-hot vector (binary class matrix). Ex: 2 -> 0010, 4 -> 1000
    def __get_label(self, item):
        return tf.keras.utils.to_categorical(self.__get_class_number(item), num_classes=self.num_classes)

    # here, we generate data containing batch_size samples
    def __get_data(self, batches):
        
        # batch for input data (images)
        x_batch = []
        for item in batches:
            img = self.__get_input(item, self.input_size)
            x_batch.append(img)
        x_batch = np.asarray(x_batch)

        # batch for labels
        y_batch = []
        for item in batches:
            label = self.__get_label(item)
            y_batch.append(label)
        y_batch = np.asarray(y_batch)

        return x_batch, y_batch

    # "entry point" of the code
    def __getitem__(self, index):
        batches = self.data_list[index * self.batch_size:(index + 1) * self.batch_size]
        x, y = self.__get_data(batches)
        return x, y

    # the number of iterations to complete an epoch
    def __len__(self):
        return self.n // self.batch_size

To use the custom generator, we have to define two lists:
*   **Training list**: contains all paths of the training images
*   **Validation list**: contains all paths of the validation images



In [ ]:
from glob import glob
from os.path import join 

# read all images
data_list = glob(join('/content/Euclid_dataset_DL', '*', '*.png'))
np.random.shuffle(data_list)

# define the percentage of the train/val
perc_train = 0.8

# actually create the two lists
train_list = data_list[:int(perc_train*len(data_list))]
val_list = data_list[int(perc_train*len(data_list)):]

print("Training data:", len(train_list))
print("Validation data:", len(val_list))

Now, we can define two instances of the class `CustomDataGen`.

In [ ]:
train_ds = CustomDataGen(data_list=train_list, num_classes=4, batch_size=32, input_size=(224, 224, 3), shuffle=True)
val_ds = CustomDataGen(data_list=val_list, num_classes=4, batch_size=32, input_size=(224, 224, 3), shuffle=True)

### Model architecture
It's time to define the architecture of our *Convolutional Neural Network* (CNN), and we have two options:

1.   Defining our **own architecture**.
2.   Using one of the architecture proposed in the literature.  

In our exercitation, we use the `MobileNet` CNN. We will use this architecture **pre-trained** on the `Imagenet` dataset (we are going to download the weights of the network from the official storage).





In [ ]:
# Our model
model = tf.keras.applications.MobileNet(
    input_shape=None,
    alpha=1.0,
    depth_multiplier=1,
    dropout=0.001,
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax"
    )

As you can see, the pre-trained model has 1000 classes, i.e. the final dense layer has 1000 neurons, while in our classification task we have only 4 classes.

We need to adapt the architecture. We can remove the last layer and create a new dense layer with only 4 neurons.

In [ ]:
# Create a layer where input is the output of the second last layer
output = Dense(4, activation='softmax', name='predictions')(model.layers[-2].output)

# Then create the corresponding model
model = Model(model.input, output)

Here we define: 

*   How many epochs
*   weights saving
*   optimizer
*   loss function
*   metric to be used to check performance of the network



In [ ]:
epochs = 5

callbacks = [
    # to save the model after every epoch
    keras.callbacks.ModelCheckpoint("save_at_{epoch}.h5"),
    # logging
    tf.keras.callbacks.TensorBoard(log_dir="logs", write_graph=True, write_images=False, update_freq="epoch",)
]

model.compile(
    optimizer=keras.optimizers.SGD(1e-3),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

### Training
If the training is too slow, remember to run the code on a machine equipped with a GPU.

On Colab, click on **Runtime → Change Runtime type → GPU** (from the drop-down menu)

We can further improve the load of images, setting workers > 1.
Object sequence is a **safer way** to do **multiprocessing**. This structure guarantees that the network **will only train once on each sample per epoch** which is not the case with generators.

In [ ]:
model.fit(
    train_ds, epochs=epochs, callbacks=callbacks, validation_data=val_ds, workers=10
)

### How to use the CNN on single images

Firstly, we have to define the function `get_class_name` to get the name of the label.

In [ ]:
def get_class_name(label):
    if label == 0:
        return 'triangle'
    elif label == 1:
        return 'rectangle'
    elif label == 2:
        return 'square'
    elif label == 3:
        return 'rhombus'
    else:
        raise(NotImplementedError)

Predict the class of a single image.

We open the image through the **OpenCV** library.

**Tools**:

*   `cv2.imread()`: open an image
*   `cv2.resize()`: resize an image
*   `np.asarray(..., dtype=...)`: convert the input to an array, with a specific data type

**NB**: **input values must be float**, not integers!



In [ ]:
import numpy as np
import cv2

# load the (trained!) model
model_trained = keras.models.load_model('/content/save_at_5.h5')

# load an image
image_path = '/content/Euclid_dataset_DL/triangle/0_triangle.png'
image_path = '/content/Euclid_dataset_DL/rhombus/0_rhombus.png'
img = cv2.imread(image_path, 1)

# REMEMBER TO APPLY THE VERY SAME PROCEDURE USED DURING TRAINING!!

# BGR -> RGB
img = img[..., ::-1]
# resize to 224x224
img = cv2.resize(img, dsize=(224, 224))
# uint8 -> float32
img = np.asarray(img, dtype=np.float32)
# image normalization
img /= 255.

# check the image shape
print(img.shape)
# add one dimension (the batch at the "beginning")
img = np.expand_dims(img, 0)
# check the final shape
print(img.shape)

# predict
prediction = model_trained.predict(img)

print('Predicted: {}'.format(prediction))
print('Predicted: {}'.format(get_class_name(np.argmax(prediction[0]))))